## Reading Data

Relevant imports go here:

In [289]:
import pandas as pd
import numpy as np
from IPython.display import display

Read in the different csv files

In [290]:
import pandas as pd

routes_df = pd.read_csv('data_output/routes.csv')
countries_df = pd.read_csv('data_output/countries.csv')
planes_df = pd.read_csv('data_output/planes.csv')
airports_df = pd.read_csv('data_output/airports.csv', on_bad_lines='skip')
airlines_df = pd.read_csv('data_output/airlines.csv', on_bad_lines='skip')

Inspect briefly the dataframes

In [291]:
# display(routes_df.head())
# display(planes_df.head())
# display(countries_df.head())
# display(airports_df.head())
# display(airlines_df.head())

## Data Cleaning 

### Cleaning for All Dataframes

Upon inspecting the dataframes, we find a few fixes that need to be made before proceeding into the data analysis stage. These issues + their relevant fixes will be highlighted in this section.

All the dataframes have their values in the cells within the quotation marks. Here, we will get rid of the quotation marks using the replace function with regular expressions. So, for instance, the name of an airplane will go from **"Aerospatiale (Nord) 262"** to **Aerospatiale (Nord) 262**

In [292]:
routes_df = routes_df.replace('"', '', regex=True)
planes_df = planes_df.replace('"', '', regex=True)
countries_df = countries_df.replace('"', '', regex=True)
airports_df = airports_df.replace('"', '', regex=True)
airlines_df = airlines_df.replace('"', '', regex=True)

### Cleaning for routes_df

We start by performing a general inspection of the routes_df dataframe.

In [293]:
display(routes_df.describe(include='all'))
display(routes_df.info())

,Airline,Airline ID,Source Airport,Source Airport ID,Destination Airport,Destination Airport ID,Codeshare,Stops,Equipment
count,67663,67663,67663,67663,67663,67663,14597,67663.000000,67645
unique,568,548,3409,3321,3418,3327,1,NaN,3925
top,FR,4296,ATL,3682,ATL,3682,Y,NaN,320
freq,2484,2484,915,915,911,911,14597,NaN,9180
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000163,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.012749,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67663 entries, 0 to 67662
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Airline                 67663 non-null  object
 1   Airline ID              67663 non-null  object
 2   Source Airport          67663 non-null  object
 3   Source Airport ID       67663 non-null  object
 4   Destination Airport     67663 non-null  object
 5   Destination Airport ID  67663 non-null  object
 6   Codeshare               14597 non-null  object
 7   Stops                   67663 non-null  int64 
 8   Equipment               67645 non-null  object
dtypes: int64(1), object(8)
memory usage: 4.6+ MB


None

Of the first items of interest, we find that the codeshare column is mainly a column that is filled with 'NaN' values. Further inspection upon the usefulness of the column indicates low relevance for the purpose of this analysis. The codeshare column will appear as "Y" if the particular flight is not operated by the airline but by another carrier. Therefore, we will first drop this column. 

We also drop the airline column because we can refer to the Airline ID column to reference airline names from the airline_df.

In [294]:
routes_df.drop(columns=['Airline', 'Codeshare'], axis = 1, inplace=True)

Next, we note that there are some rows in the equipment column that have null objects. Because the equipment column (which shows what types of aircrafts are mainly used for the routes) is an important one, we will omit all the rows that have null equipment values.

In [295]:
routes_df = routes_df.dropna(subset=['Equipment'])

Continuing the work on the 'equipment' column, we find that in some cases, the column has multiple equipment all listed in the same cell. For instance, take rows 64049 to 64055 as shown below.

In [296]:
routes_df.iloc[64049:64056]

,Airline ID,Source Airport,Source Airport ID,Destination Airport,Destination Airport ID,Stops,Equipment
64067,4547,MCO,3878,PHX,3462,0,73W 738
64068,4547,MCO,3878,PIT,3570,0,73W
64069,4547,MCO,3878,PVD,3641,0,73C 733 73W 73H
64070,4547,MCO,3878,RDU,3626,0,73W 733 73C
64071,4547,MCO,3878,RIC,3608,0,733 73W 73C
64072,4547,MCO,3878,ROC,3622,0,73W
64073,4547,MCO,3878,SAT,3621,0,73H 73W


As shown, some of the cells in the 'Equipment' column have more than one equipment/aircraft type listed within the cell. We would like to have a separate row for each of the equipment in order for appropriate analysis. Therefore, we need to separate a row which has the cell "73C 733 73W 73H" into four different rows so that every type of equipment has its own row. The rows would be: 
- Row 1 would have the 73C in the 'Equipment' column
- Row 2 would have the 733 in the 'Equipment' column
- Row 3 would have the 73W in the 'Equipment' column
- Row 4 would have the 73H in the 'Equipment' column

Therefore, the last step in cleaning the routes_df would be to 'explode' these rows with multiple strings in the 'Equipment' column into different rows for each equipment. 

In [297]:
routes_df['Equipment'] = routes_df['Equipment'].str.split()
routes_df = routes_df.explode('Equipment')

Because the routes_df Airline ID, Source Airport ID, Destination Airport ID will be important in performing merges in the latter part of the analysis, we will need drop all rows with '\N' or empty values for each of these columns. 

In [298]:
routes_df = routes_df[~((routes_df['Airline ID'] == '') | (routes_df['Airline ID']== '\\N'))]
routes_df = routes_df[~((routes_df['Source Airport ID'] == '') | (routes_df['Source Airport ID']== '\\N'))]
routes_df = routes_df[~((routes_df['Destination Airport ID'] == '') | (routes_df['Destination Airport ID']== '\\N'))]

We also need to ensure that the columns with numbers in them are converted into int type!

In [299]:
routes_df['Airline ID'] = routes_df['Airline ID'].astype(int)
routes_df['Source Airport ID'] = routes_df['Source Airport ID'].astype(int)
routes_df['Destination Airport ID'] = routes_df['Destination Airport ID'].astype(int)

### Cleaning for countries_df

We start by performing a general inspection of the countries_df dataframe.

In [300]:
display(countries_df.describe(include='all'))
display(countries_df.info())

,name,iso_code,dafif_code,Unnamed: 3
count,261,261,261,2
unique,259,241,260,2
top,India,\N,BQ,
freq,2,19,2,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261 entries, 0 to 260
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        261 non-null    object
 1   iso_code    261 non-null    object
 2   dafif_code  261 non-null    object
 3   Unnamed: 3  2 non-null      object
dtypes: object(4)
memory usage: 8.3+ KB


None

Whilst inspecting the countries_df, we found a few items of interest. First, we find that there are 261 rows in the countries dataframe, but only 259 are unique (as seen in the unique column). Let's inspect further. 

In [301]:
duplicate_countries = countries_df[countries_df.duplicated(subset='name', keep=False)]
print(duplicate_countries)

          name iso_code dafif_code Unnamed: 3
33       India       IN         BS        NaN
101  Palestine       PS         GZ        NaN
112      India       IN         IN        NaN
252  Palestine       PS         WE        NaN


The reason for the duplicates is because the country 'India' and 'Palestine; has two different dafif_codes. Further research shows that the "DAFIF code" is not a standard or widely recognized code for countries. It's possible that "DAFIF" refers to a specific system or dataset that uses its own set of codes for countries or locations. DAFIF (Digital Aeronautical Flight Information File) codes are mainly used in the aviation and aeronautical navigation field and the information is not readily available. Seeing that the dafif_code is not as popular, and that the ISO code is more standard, this will guide us to focus on the ISO_codes from now. For the purpose of cleaning and avoiding duplicates, we will get rid of the first of the duplicates for each of the duplicated countries.

The reason for the duplicates is because the country 'India' has two different dafif_codes. The correct dafif_code for India is "IN". Therefore, we will drop the first of the two rows in india_rows (index = 33). 

In [302]:
countries_df = countries_df.drop_duplicates(subset=['name'], keep='last')

We also note that the discrepancy in ISO_codes where, there are 241 unique ISO codes and 259 (now) unique countries. Referring back to the guide from the website that provides the dataset, we find the following "Some entries have DAFIF codes, but not ISO codes. These are primarily uninhabited islands without airports, and can be ignored for most purposes." Given that these locations do not have airports, they become useless for the purpose of our research. Therefore, we will omit all the rows where iso_code == '\N'.

In [303]:
countries_df = countries_df[countries_df['iso_code'] != '\\N']

Now that we have cleaned up the countries_df, we go back to obsolete nature of the dafif_code column and the "Unnamed: 3" column. The final step in making this dataframe analysis-ready is to drop the irrelevant columns. We drop the last two columns of the dataframe. 

Some entries have DAFIF codes, but not ISO codes. These are primarily uninhabited islands without airports, and can be ignored for most purposes.

In [304]:
countries_df = countries_df.iloc[:, :-2]

## Cleaning for planes_df

Before delving into the cleaning for the planes_df, we would like to clarify the following:

- The original planes_df only had the 'Name', 'IATA Code' and 'ICAO code' columns. Upon a visual inspection of the csv file, we noticed that for some of the fields (mainly the smaller/uncommon planes), the IATA or ICAO code was missing. Whilst the number of missing IATA/ICAO codes missing were not significant enough for us to be worried about the completeness data, we handled the missing information by looking the missing data points and filling them in manually. This would be the most direct and time-efficient solution to employ.
- The 'General Make' column was also manually created mainly to accommodate for the Boeings and Airbuses. Acknowledging the huge duopoly that Boeing and Airbus have in terms of airplanes, we are cognizant that a significant number of comparisons and flights being analysed will pertain to aircrafts manufactured by either of the companies. By creating the 'General Make' column, we would like to appropriately account for the many variations of a single model of an aircraft that both companies create.
   - For instance, the Boeing 737 comes in several variations: Boeing 737 MAX 10, Boeing 737 MAX 7, Boeing 737 MAX 8, Boeing 737 MAX 9, Boeing 737-200, Boeing 737-300, Boeing 737-400, Boeing 737-500, etc. Whilst the variations provide a significant amount of detail, we recognize that it may help to group them all as "B737". Similar can be said for the Airbus340 which comes in 5 different makes therefore, the ICAO of the Airbus340 is "A340" which is what will be used to fill in all of the rows that have an Airbus340 make. The goal of the "General Make" is to avoid dwelling on the specifics of the model and rather recognize the general model. For other aircraft makes of aircrafts (for instance Lockheed Martin, Aerospatiale, etc), if the makes shared various types of the model, we also labelled them similarly (based on the main model). If an aircraft is the only single make in the model type, we use the details from the "ICAO code" column and copy it to the "General Make" column.
- For some airplane types, especially the really small/uncommon airplanes, we were unable to find the right IATA codes. Therefore, we have left those rows as is. It is very unlikely that airplanes that do not have an IATA code will be utilized as transportation vehicles on many flight routes anyway.


With this, let us take a look at the planes_df briefly.

In [305]:
planes_df.head()

,Name,IATA Code,ICAO Code,General Make
0,Aerospatiale (Nord) 262,ND2,N262,N262
1,Aerospatiale (Sud Aviation) Se.210 Caravelle,CRV,S210,S210
2,Aerospatiale SN.601 Corvette,NDC,S601,S601
3,Aerospatiale/Alenia ATR 42-300,AT4,AT43,AT43
4,Aerospatiale/Alenia ATR 42-500,AT5,AT45,AT45


In [306]:
display(planes_df.describe(include='all'))
display(planes_df.info())

,Name,IATA Code,ICAO Code,General Make
count,246,246,246,246
unique,246,223,243,186
top,Aerospatiale (Nord) 262,\N,B737,B737
freq,1,9,3,13


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246 entries, 0 to 245
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Name          246 non-null    object
 1   IATA Code     246 non-null    object
 2   ICAO Code     246 non-null    object
 3   General Make  246 non-null    object
dtypes: object(4)
memory usage: 7.8+ KB


None

From the tables, the item that grabs our attention immediately is the number of unique IATA codes. There are some empty cells for the IATA code of smaller/uncommon aircrafts because they do not have an IATA code. To handle these, we can change the '\N' cells in the IATA column into cells with null values. 

In [307]:
planes_df['IATA Code'] = planes_df['IATA Code'].replace(r'\N', np.nan)
display(planes_df.describe(include='all'))
display(planes_df.info())

,Name,IATA Code,ICAO Code,General Make
count,246,237,246,246
unique,246,222,243,186
top,Aerospatiale (Nord) 262,CNJ,B737,B737
freq,1,8,3,13


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246 entries, 0 to 245
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Name          246 non-null    object
 1   IATA Code     237 non-null    object
 2   ICAO Code     246 non-null    object
 3   General Make  246 non-null    object
dtypes: object(4)
memory usage: 7.8+ KB


None

We notice that the names for the airplanes are really long. Whilst this is good because it gives us some detail, we would also like to do a comparison of the Boeings vs Airbus vs other airplanes. For this, we will create a new column that takes the first string of the "Name" column which is most often the name of the company. 

In [308]:
planes_df['Company'] = planes_df['Name'].str.split().str[0]

To make the analysis more analysis-ready, we can add a new column acknowledging that most of the airplanes for routes will be Boeings and Airbuses. We can make a new column called "airbus_boeing_other" which allocates all Airbus airplanes from the "Company" column as "Airbus" and labels all the Boeing aircrafts as "Boeing". However, for all of the other aircrafts that are not Boeings and Airbuses, these would be labelled as "Other".

In [309]:
planes_df['airbus_boeing_other'] = np.where(planes_df['Company'].isin(['Boeing', 'Airbus']), planes_df['Company'], 'Other')
display(planes_df['airbus_boeing_other'].value_counts())

airbus_boeing_other
Other     172
Boeing     47
Airbus     27
Name: count, dtype: int64

planes_df is now analysis ready! 

## Cleaning for airports_df

In analysing the airports_df, which contains a list of all the airports in 2014, we start by using our describe and info functions to get an overview of the dataset.

In [310]:
display(airports_df.describe(include='all'))
display(airports_df.info())

,ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,TzDatabase,Type,Source
count,7678.000000,7678,7678,7678,7678,7678,7678.000000,7678.000000,7678.000000,7678,7678,7678,7678,7678
unique,NaN,7638,6938,237,6055,7678,NaN,NaN,NaN,41,8,308,1,1
top,NaN,San Pedro Airport,,United States,\N,AYGA,NaN,NaN,NaN,1,U,\N,airport,OurAirports
freq,NaN,3,49,1510,1624,1,NaN,NaN,NaN,1174,1859,1020,7678,7678
mean,5173.247591,NaN,NaN,NaN,NaN,NaN,25.780464,-1.419112,1017.946080,NaN,NaN,NaN,NaN,NaN
std,3778.845117,NaN,NaN,NaN,NaN,NaN,28.376413,86.548631,1630.316793,NaN,NaN,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN,NaN,-90.000000,-179.876999,-1266.000000,NaN,NaN,NaN,NaN,NaN
25%,1993.250000,NaN,NaN,NaN,NaN,NaN,6.844542,-79.006977,63.000000,NaN,NaN,NaN,NaN,NaN
50%,4066.500000,NaN,NaN,NaN,NaN,NaN,34.066328,6.317980,354.000000,NaN,NaN,NaN,NaN,NaN
75%,7736.500000,NaN,NaN,NaN,NaN,NaN,47.206125,56.139001,1206.000000,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7678 entries, 0 to 7677
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          7678 non-null   int64  
 1   Name        7678 non-null   object 
 2   City        7678 non-null   object 
 3   Country     7678 non-null   object 
 4   IATA        7678 non-null   object 
 5   ICAO        7678 non-null   object 
 6   Latitude    7678 non-null   float64
 7   Longitude   7678 non-null   float64
 8   Altitude    7678 non-null   int64  
 9   Timezone    7678 non-null   object 
 10  DST         7678 non-null   object 
 11  TzDatabase  7678 non-null   object 
 12  Type        7678 non-null   object 
 13  Source      7678 non-null   object 
dtypes: float64(2), int64(2), object(10)
memory usage: 839.9+ KB


None

By looking at the output, we observe a few things: 
- The type column does not hold any useful information. It mentions "airport" for all the 7678 airports, which as is known, are all airports
- The source column tells us where the information is coming from. For all of the rows, the source is the same which means no new information is being held in this column.
- The 'TzDatabase' column has 1020 null values. Upon further research we see that the column shares the timezone information in Olson format, eg. "America/Los_Angeles". We also realize that all the other columns related to the timezone and DST are not going to be utilized for this analysis. We drop all these columns. 
- The IATA code has many null values. Since the ICAO code (which is more complete) can serve as the unique identifier for this column, the IATA code column has no value.
- There are 7678 rows in the dataset but only 7638 of the airports are unique. We will need to look into these further.

We can first start by dropping the columns which will not benefit our analysis.

In [311]:
airports_df.drop(['Source', 'Type', 'TzDatabase', 'IATA', 'Timezone', 'DST'], axis=1, inplace=True)

Now that we have dropped the unnecessary columns, we can inspect the 'Name' column and observe why we have some 7638 unique airports of 7678 total airports. 

In [312]:
duplicate_airports = airports_df[airports_df.duplicated(subset='Name', keep=False)]

In [313]:
duplicate_airports

,ID,Name,City,Country,ICAO,Latitude,Longitude,Altitude
44,45,Deer Lake Airport,Deer Lake,Canada,CYDF,49.210800,-57.391399,72
180,182,Charlottetown Airport,Charlottetown,Canada,CYYG,46.290001,-63.121101,160
203,205,Faro Airport,Faro,Canada,CZFA,62.207500,-133.376007,2351
255,258,San Pedro Airport,San Pedro,Cote d'Ivoire,DISP,4.746720,-6.660820,26
516,521,Newcastle Airport,Newcastle,United Kingdom,EGNT,55.037498,-1.691670,266
...,...,...,...,...,...,...,...,...
7034,11816,Columbus Municipal Airport,Columbus,United States,KBAK,39.261902,-85.896301,656
7065,11847,Columbus Municipal Airport,Columbus,United States,KOLU,41.448002,-97.342598,1447
7154,11936,Victoria Airport,Victoria,Chile,SCTO,-38.245602,-72.348602,1148
7419,13341,Charlottetown Airport,Charlottetown,Canada,CCH4,52.764999,-56.115601,209


In [314]:
display(duplicate_airports.describe(include='all'))
display(duplicate_airports.info())

,ID,Name,City,Country,ICAO,Latitude,Longitude,Altitude
count,76.000000,76,76,76,76,76.000000,76.000000,76.000000
unique,NaN,36,57,27,76,NaN,NaN,NaN
top,NaN,Capital City Airport,Newcastle,United States,CYDF,NaN,NaN,NaN
freq,NaN,3,3,21,1,NaN,NaN,NaN
mean,5363.539474,NaN,NaN,NaN,NaN,14.070067,-32.110879,777.552632
std,3494.156785,NaN,NaN,NaN,NaN,30.036343,90.021406,1453.304514
min,45.000000,NaN,NaN,NaN,NaN,-40.091702,-177.380997,0.000000
25%,2627.000000,NaN,NaN,NaN,NaN,-10.881996,-84.676174,29.750000
50%,4926.000000,NaN,NaN,NaN,NaN,19.336650,-66.365061,276.500000
75%,7940.500000,NaN,NaN,NaN,NaN,39.421625,-5.418532,715.250000


<class 'pandas.core.frame.DataFrame'>
Index: 76 entries, 44 to 7627
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         76 non-null     int64  
 1   Name       76 non-null     object 
 2   City       76 non-null     object 
 3   Country    76 non-null     object 
 4   ICAO       76 non-null     object 
 5   Latitude   76 non-null     float64
 6   Longitude  76 non-null     float64
 7   Altitude   76 non-null     int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 5.3+ KB


None

We acknowledge and see that there are indeed duplicate values. The only issue with eliminating duplicates is that each of the rows has an 'ID' associated with it. And the routes_df above references the ID column in the airports_df by having columns for 'source ID' and 'destination airport ID'. If any of the rows from routes_df references the ID's (albeit duplicate) that we may delete right now, we might not be able to obtain a proper merge. This is especially important because the main point of our concern following the merge will be the country that the routes fly out from and not so much the intricacies like the airport or the altitude, or ICAO. For these duplicated values we find that they are duplicated due to differences in the intricacies (longitudes, ICAOs, etc) but the countries remain consistent - see for instance Arlington Municipal Airport, or Charlottetown Airport. For some airports, we realize that they are not infact duplicates, rather, they are two different airports that they have the same - see for instance Flinders Island Airport, Capital City Airport, etc. Therefore, we will leave these as is and bank mainly on the ID column to be the reference column for performing merges.

In [315]:
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None) 
duplicate_airports_sorted = duplicate_airports.sort_values(by='Name', ascending=True)
display(duplicate_airports_sorted)

,ID,Name,City,Country,ICAO,Latitude,Longitude,Altitude
5804,7846,Arlington Municipal Airport,Arlington,United States,KGKY,32.663898,-97.094299,628
6857,11141,Arlington Municipal Airport,Arlington,United States,KAWO,48.160702,-122.158997,142
4233,5538,Bathurst Airport,Bathurst,Canada,CZBF,47.629700,-65.738899,193
4863,6249,Bathurst Airport,Bathurst,Australia,YBTH,-33.409401,149.651993,2435
6225,8631,Capital City Airport,Frankfort,United States,KFFT,38.182499,-84.904701,806
6044,8284,Capital City Airport,Harrisburg,United States,KCXY,40.217098,-76.851501,347
3336,3544,Capital City Airport,Lansing,United States,KLAN,42.778702,-84.587402,861
180,182,Charlottetown Airport,Charlottetown,Canada,CYYG,46.290001,-63.121101,160
7419,13341,Charlottetown Airport,Charlottetown,Canada,CCH4,52.764999,-56.115601,209
7065,11847,Columbus Municipal Airport,Columbus,United States,KOLU,41.448002,-97.342598,1447


In [316]:
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

In [317]:
airports_df

,ID,Name,City,Country,ICAO,Latitude,Longitude,Altitude
0,1,Goroka Airport,Goroka,Papua New Guinea,AYGA,-6.081690,145.391998,5282
1,2,Madang Airport,Madang,Papua New Guinea,AYMD,-5.207080,145.789001,20
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,AYMH,-5.826790,144.296005,5388
3,4,Nadzab Airport,Nadzab,Papua New Guinea,AYNZ,-6.569803,146.725977,239
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,AYPY,-9.443380,147.220001,146
...,...,...,...,...,...,...,...,...
7673,14106,Rogachyovo Air Base,Belaya,Russia,ULDA,71.616699,52.478298,272
7674,14107,Ulan-Ude East Airport,Ulan Ude,Russia,XIUW,51.849998,107.737999,1670
7675,14108,Krechevitsy Air Base,Novgorod,Russia,ULLK,58.625000,31.385000,85
7676,14109,Desierto de Atacama Airport,Copiapo,Chile,SCAT,-27.261200,-70.779198,670


With this, our airports_df dataframe is now analysis ready!

## Cleaning for airlines_df

In analysing the airports_df, which contains a list of all the airports in 2014, we start by using our describe and info functions to get an overview of the dataset.

In [318]:
display(airlines_df.describe(include='all'))
display(airlines_df.info())

,Airline ID,Name,Alias,IATA,ICAO,Callsign,Country,Active
count,6161.000000,6161,6161,6161,6161,6161,6161,6161
unique,NaN,6072,181,1123,5855,5263,278,3
top,NaN,National Airlines,\N,,\N,,United States,N
freq,NaN,5,5478,4625,188,807,1099,4905
mean,4150.130985,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,4503.076635,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,-1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,1541.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,3082.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,4628.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6161 entries, 0 to 6160
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Airline ID  6161 non-null   int64 
 1   Name        6161 non-null   object
 2   Alias       6161 non-null   object
 3   IATA        6161 non-null   object
 4   ICAO        6161 non-null   object
 5   Callsign    6161 non-null   object
 6   Country     6161 non-null   object
 7   Active      6161 non-null   object
dtypes: int64(1), object(7)
memory usage: 385.2+ KB


None

- Immediately, looking at the summary, we see something very amusing! In the 'active' column there are 6161 total airlines however the most frequent value in the column is 'N' with a frequency of 4905 'N' values! The active column is one that is Y (yes) for active airlines and N (No) for inactive airlines
- We also see that in the same column, there are 3 unique values, which we know should Y, N but we are not sure what the last value is that is unique.

We can start by analysing these two and looking at the summary once we have dropped the inactive rows.

In [319]:
airlines_df['Active'].unique()

array(['Y', 'N', 'n'], dtype=object)

We will drop all rows with 'N' and 'n'.

In [320]:
airlines_df = airlines_df[~airlines_df['Active'].isin(['N', 'n'])]
airlines_df

,Airline ID,Name,Alias,IATA,ICAO,Callsign,Country,Active
0,-1,Unknown,\N,-,N/A,\N,\N,Y
1,1,Private flight,\N,-,N/A,,,Y
3,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
10,10,40-Mile Air,\N,Q5,MLA,MILE-AIR,United States,Y
13,13,Ansett Australia,\N,AN,AAA,ANSETT,Australia,Y
...,...,...,...,...,...,...,...,...
6153,21179,Thai Vietjet Air,,,TVJ,THAIVIET JET,Thailand,Y
6156,21248,GX Airlines,,,CBG,SPRAY,China,Y
6158,21268,Jetgo Australia,,JG,\N,,Australia,Y
6159,21270,Air Carnival,,2S,\N,,India,Y


In [321]:
display(airlines_df.describe(include='all'))
display(airlines_df.info())

,Airline ID,Name,Alias,IATA,ICAO,Callsign,Country,Active
count,1255.000000,1255,1255,1255,1255,1255,1255,1255
unique,NaN,1249,152,996,1087,920,197,1
top,NaN,Thomas Cook Airlines,\N,,\N,,United States,Y
freq,NaN,2,712,239,134,323,156,1255
mean,8201.240637,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,7136.125474,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,-1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,2255.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,4687.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,16123.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
Index: 1255 entries, 0 to 6160
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Airline ID  1255 non-null   int64 
 1   Name        1255 non-null   object
 2   Alias       1255 non-null   object
 3   IATA        1255 non-null   object
 4   ICAO        1255 non-null   object
 5   Callsign    1255 non-null   object
 6   Country     1255 non-null   object
 7   Active      1255 non-null   object
dtypes: int64(1), object(7)
memory usage: 88.2+ KB


None

For the purposes of this analysis we realize that the IATA, ICAO, Alias and Callsign columns are all irrelevant. Therefore, we can drop these too. 

In [322]:
airlines_df.drop(['Alias', 'IATA', 'ICAO', 'Callsign'], axis=1, inplace=True)
airlines_df

,Airline ID,Name,Country,Active
0,-1,Unknown,\N,Y
1,1,Private flight,,Y
3,3,1Time Airline,South Africa,Y
10,10,40-Mile Air,United States,Y
13,13,Ansett Australia,Australia,Y
...,...,...,...,...
6153,21179,Thai Vietjet Air,Thailand,Y
6156,21248,GX Airlines,China,Y
6158,21268,Jetgo Australia,Australia,Y
6159,21270,Air Carnival,India,Y


We would also like to drop the rows with no value or '\N' in the Country column

In [323]:
airlines_df = airlines_df[~((airlines_df['Country'] == '') | (airlines_df['Country'] == '\\N'))]
airlines_df

,Airline ID,Name,Country,Active
3,3,1Time Airline,South Africa,Y
10,10,40-Mile Air,United States,Y
13,13,Ansett Australia,Australia,Y
14,14,Abacus International,Singapore,Y
21,21,Aigle Azur,France,Y
...,...,...,...,...
6153,21179,Thai Vietjet Air,Thailand,Y
6156,21248,GX Airlines,China,Y
6158,21268,Jetgo Australia,Australia,Y
6159,21270,Air Carnival,India,Y


Now that we have ensured that all of the airlines are active, we can also drop the active column since it presents no new information

In [324]:
airlines_df.drop(['Active'], axis=1, inplace=True)

/var/folders/8q/ncnhj12n3p328_9h8_4j1y6h0000gn/T/ipykernel_42732/4088524110.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airlines_df.drop(['Active'], axis=1, inplace=True)


The airlines_df is now analysis ready!

## Merging the datasets

We would now like to merge these datasets in order to be of use when analyzing in order to draw out patterns and conclusions. We will start by merging the routes_df to the airlines_df.

But before we merge these datasets, we need to ensure that the names of the columns in both the dataframes are unique and understandable once the merge takes place!

In [325]:
routes_df.columns

Index(['Airline ID', 'Source Airport', 'Source Airport ID',
       'Destination Airport', 'Destination Airport ID', 'Stops', 'Equipment'],
      dtype='object')

In [326]:
routes_df.columns = ['airline_ID', 'route_source_airport', 'route_source_ID',
                     'route_destination_airport', 'route_destination_ID', 'route_stops', 'route_equipment']

In [327]:
airlines_df.columns

Index(['Airline ID', 'Name', 'Country'], dtype='object')

In [328]:
airlines_df.columns = ['airline_ID', 'airline_name', 'airline_country']

We now need to make sure that airline_ID in both of the dataframes are int. 

We can now perform our first merge for routes_df with airlines_df where we will merge route_airline_ID from the routes_df and the airline_ID from the airlines_df. 

routes_df['airline_ID'] = routes_df['airline_ID'].astype(int)
airlines_df['airline_ID'] = airlines_df['airline_ID'].astype(int)

In [329]:
merged_df = routes_df.merge(airlines_df, on='airline_ID', how='left')
merged_df

,airline_ID,route_source_airport,route_source_ID,route_destination_airport,route_destination_ID,route_stops,route_equipment,airline_name,airline_country
0,410,AER,2965,KZN,2990,0,CR2,Aerocondor,Portugal
1,410,ASF,2966,KZN,2990,0,CR2,Aerocondor,Portugal
2,410,ASF,2966,MRV,2962,0,CR2,Aerocondor,Portugal
3,410,CEK,2968,KZN,2990,0,CR2,Aerocondor,Portugal
4,410,CEK,2968,OVB,4078,0,CR2,Aerocondor,Portugal
...,...,...,...,...,...,...,...,...,...
92211,4178,WYA,6334,ADL,3341,0,SF3,Regional Express,Australia
92212,19016,DME,4029,FRU,2912,0,734,Apache Air,United States
92213,19016,FRU,2912,DME,4029,0,734,Apache Air,United States
92214,19016,FRU,2912,OSS,2913,0,734,Apache Air,United States


Amazing! We can now move into merging the merged_df with the airports_df. Because we have both route_source_ID and route_destination_ID, we need to pick the one we would merge onto. We will start by renaming the columns on the airports_df for ease of understanding

In [330]:
airports_df.columns

Index(['ID', 'Name', 'City', 'Country', 'ICAO', 'Latitude', 'Longitude',
       'Altitude'],
      dtype='object')

In [331]:
airports_df.columns = ['airport_ID', 'airport_name', 'airport_city', 'airport_country', 'airport_ICAO', 'airport_lat', 'airport_long', 'airport_alt']

For the purpose of continuity and ease of understanding, we will first merge it with the route_destination_ID. To do so, and to avoid confusion, we will first duplicate the route_destination_ID and give it a name that will allow for a merge with the airports_df.

We will duplicate the airport_ID column and call it route_source_ID so we can merge it with the route_source_ID from the merged_df

In [332]:
airports_df['route_source_ID'] = airports_df['airport_ID']

In [333]:
merged_df = merged_df.merge(airports_df, on='route_source_ID', how='left')

Because we still have to merge the destination airport information, we will need to rename the columns to accommodate for what is to come and to avoid any confusion. 

In [334]:
merged_df

,airline_ID,route_source_airport,route_source_ID,route_destination_airport,route_destination_ID,route_stops,route_equipment,airline_name,airline_country,airport_ID,airport_name,airport_city,airport_country,airport_ICAO,airport_lat,airport_long,airport_alt
0,410,AER,2965,KZN,2990,0,CR2,Aerocondor,Portugal,2965.0,Sochi International Airport,Sochi,Russia,URSS,43.449902,39.956600,89.0
1,410,ASF,2966,KZN,2990,0,CR2,Aerocondor,Portugal,2966.0,Astrakhan Airport,Astrakhan,Russia,URWA,46.283298,48.006302,-65.0
2,410,ASF,2966,MRV,2962,0,CR2,Aerocondor,Portugal,2966.0,Astrakhan Airport,Astrakhan,Russia,URWA,46.283298,48.006302,-65.0
3,410,CEK,2968,KZN,2990,0,CR2,Aerocondor,Portugal,2968.0,Chelyabinsk Balandino Airport,Chelyabinsk,Russia,USCC,55.305801,61.503300,769.0
4,410,CEK,2968,OVB,4078,0,CR2,Aerocondor,Portugal,2968.0,Chelyabinsk Balandino Airport,Chelyabinsk,Russia,USCC,55.305801,61.503300,769.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92211,4178,WYA,6334,ADL,3341,0,SF3,Regional Express,Australia,6334.0,Whyalla Airport,Whyalla,Australia,YWHA,-33.058899,137.514008,41.0
92212,19016,DME,4029,FRU,2912,0,734,Apache Air,United States,4029.0,Domodedovo International Airport,Moscow,Russia,UUDD,55.408798,37.906300,588.0
92213,19016,FRU,2912,DME,4029,0,734,Apache Air,United States,2912.0,Manas International Airport,Bishkek,Kyrgyzstan,UAFM,43.061298,74.477600,2058.0
92214,19016,FRU,2912,OSS,2913,0,734,Apache Air,United States,2912.0,Manas International Airport,Bishkek,Kyrgyzstan,UAFM,43.061298,74.477600,2058.0


In [335]:
merged_df.columns

Index(['airline_ID', 'route_source_airport', 'route_source_ID',
       'route_destination_airport', 'route_destination_ID', 'route_stops',
       'route_equipment', 'airline_name', 'airline_country', 'airport_ID',
       'airport_name', 'airport_city', 'airport_country', 'airport_ICAO',
       'airport_lat', 'airport_long', 'airport_alt'],
      dtype='object')

In [336]:
merged_df.columns = (['airline_ID', 'route_source_airport', 'route_source_ID',
       'route_destination_airport', 'route_destination_ID', 'route_stops',
       'route_equipment', 'airline_name', 'airline_country', 'source_airport_ID',
       'source_airport_name', 'source_airport_city', 'source_airport_country', 'source_airport_ICAO',
       'source_airport_lat', 'source_airport_long', 'source_airport_alt'])

The next step is to merge the destination airport information. Similar to what we did for the airports_df in terms of duplicating the airport_ID column to make it an easy merge, we will do the same again but for the purposes of merging on the route_destination_ID. Because we already have a route_source_ID column in the airports_df which we will not use anymore, we will just rename it!

In [337]:
airports_df.columns

Index(['airport_ID', 'airport_name', 'airport_city', 'airport_country',
       'airport_ICAO', 'airport_lat', 'airport_long', 'airport_alt',
       'route_source_ID'],
      dtype='object')

In [338]:
airports_df.rename(columns={'route_source_ID': 'route_destination_ID'}, inplace=True)

And, we will now perform the merge to incorporate the destination airports information!

In [339]:
merged_df = merged_df.merge(airports_df, on='route_destination_ID', how='left')

Just like the previous merge, we will rename the columns for ease of understanding and following

In [340]:
merged_df.columns

Index(['airline_ID', 'route_source_airport', 'route_source_ID',
       'route_destination_airport', 'route_destination_ID', 'route_stops',
       'route_equipment', 'airline_name', 'airline_country',
       'source_airport_ID', 'source_airport_name', 'source_airport_city',
       'source_airport_country', 'source_airport_ICAO', 'source_airport_lat',
       'source_airport_long', 'source_airport_alt', 'airport_ID',
       'airport_name', 'airport_city', 'airport_country', 'airport_ICAO',
       'airport_lat', 'airport_long', 'airport_alt'],
      dtype='object')

In [341]:
merged_df.columns = (['airline_ID', 'route_source_airport', 'route_source_ID',
       'route_destination_airport', 'route_destination_ID', 'route_stops',
       'route_equipment', 'airline_name', 'source_airline_country',
       'source_airport_ID', 'source_airport_name', 'source_airport_city',
       'source_airport_country', 'source_airport_ICAO', 'source_airport_lat',
       'source_airport_long', 'source_airport_alt', 'destination_airport_ID',
       'destination_airport_name', 'destination_airport_city', 'destination_airport_country', 'destination_airport_ICAO',
       'destination_airport_lat', 'destination_airport_long', 'destination_airport_alt'])

Amazing! Now that we have a lot of columns, we realize that we can get rid of some without impacting the ability to do our further analysis. For instance, we recognize that there are two columns that hold the ID's for both: the source and the destination. We also realize that the ICAO columns do not bring any value. The route_source_airport and route_destination_airports also do not bring much value now that we have the full names for the airports. Therefore, before proceeding, we will get rid of these.

In [342]:
merged_df.drop(['route_source_airport', 'route_destination_airport', 'route_source_ID', 'route_destination_ID', 'source_airport_ICAO', 'destination_airport_ICAO'], axis=1, inplace=True)

In [343]:
merged_df.columns

Index(['airline_ID', 'route_stops', 'route_equipment', 'airline_name',
       'source_airline_country', 'source_airport_ID', 'source_airport_name',
       'source_airport_city', 'source_airport_country', 'source_airport_lat',
       'source_airport_long', 'source_airport_alt', 'destination_airport_ID',
       'destination_airport_name', 'destination_airport_city',
       'destination_airport_country', 'destination_airport_lat',
       'destination_airport_long', 'destination_airport_alt'],
      dtype='object')

To merge the merged_df with the planes_df, we take interest in the 'equipment' column of merged_df. The 'equipment' column is matched with the 'IATA' code in the planes_df. To enable the merge, we will first rename the column in the merged_df to 'IATA'

In [348]:
merged_df.rename(columns={'route_equipment': 'IATA Code'}, inplace=True)

In [351]:
merged_df = merged_df.merge(planes_df, on='IATA Code', how='left')

Once again, we will rename the columns in order to easily track the columns and their purposes

In [354]:
merged_df.columns = (['airline_ID', 'route_stops', 'IATA Code', 'airline_name',
       'source_airline_country', 'source_airport_ID', 'source_airport_name',
       'source_airport_city', 'source_airport_country', 'source_airport_lat',
       'source_airport_long', 'source_airport_alt', 'destination_airport_ID',
       'destination_airport_name', 'destination_airport_city',
       'destination_airport_country', 'destination_airport_lat',
       'destination_airport_long', 'destination_airport_alt', 'aircraft_name',
       'icao_code', 'aircraft_general_make', 'aircraft_company', 'airbus_boeing_other'])

In [356]:
new_column_order = ['source_airline_country', 'source_airport_ID', 'source_airport_name',
       'source_airport_city', 'source_airport_country', 'source_airport_lat',
       'source_airport_long', 'source_airport_alt', 'destination_airport_ID',
       'destination_airport_name', 'destination_airport_city',
       'destination_airport_country', 'destination_airport_lat',
       'destination_airport_long', 'destination_airport_alt', 'airline_ID', 'airline_name', 'route_stops', 'IATA Code',
       'icao_code', 'aircraft_name','aircraft_general_make', 'aircraft_company', 'airbus_boeing_other']

sorted_df = merged_df[new_column_order]

In [357]:
sorted_df

,source_airline_country,source_airport_ID,source_airport_name,source_airport_city,source_airport_country,source_airport_lat,source_airport_long,source_airport_alt,destination_airport_ID,destination_airport_name,...,destination_airport_alt,airline_ID,airline_name,route_stops,IATA Code,icao_code,aircraft_name,aircraft_general_make,aircraft_company,airbus_boeing_other
0,Portugal,2965.0,Sochi International Airport,Sochi,Russia,43.449902,39.956600,89.0,2990.0,Kazan International Airport,...,411.0,410,Aerocondor,0,CR2,CRJ2,Canadair Regional Jet 200,CRJ2,Canadair,Other
1,Portugal,2966.0,Astrakhan Airport,Astrakhan,Russia,46.283298,48.006302,-65.0,2990.0,Kazan International Airport,...,411.0,410,Aerocondor,0,CR2,CRJ2,Canadair Regional Jet 200,CRJ2,Canadair,Other
2,Portugal,2966.0,Astrakhan Airport,Astrakhan,Russia,46.283298,48.006302,-65.0,2962.0,Mineralnyye Vody Airport,...,1054.0,410,Aerocondor,0,CR2,CRJ2,Canadair Regional Jet 200,CRJ2,Canadair,Other
3,Portugal,2968.0,Chelyabinsk Balandino Airport,Chelyabinsk,Russia,55.305801,61.503300,769.0,2990.0,Kazan International Airport,...,411.0,410,Aerocondor,0,CR2,CRJ2,Canadair Regional Jet 200,CRJ2,Canadair,Other
4,Portugal,2968.0,Chelyabinsk Balandino Airport,Chelyabinsk,Russia,55.305801,61.503300,769.0,4078.0,Tolmachevo Airport,...,365.0,410,Aerocondor,0,CR2,CRJ2,Canadair Regional Jet 200,CRJ2,Canadair,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92354,Australia,6334.0,Whyalla Airport,Whyalla,Australia,-33.058899,137.514008,41.0,3341.0,Adelaide International Airport,...,20.0,4178,Regional Express,0,SF3,SF34,Saab SF340A/B,SF34,Saab,Other
92355,United States,4029.0,Domodedovo International Airport,Moscow,Russia,55.408798,37.906300,588.0,2912.0,Manas International Airport,...,2058.0,19016,Apache Air,0,734,B734,Boeing 737-400,B737,Boeing,Boeing
92356,United States,2912.0,Manas International Airport,Bishkek,Kyrgyzstan,43.061298,74.477600,2058.0,4029.0,Domodedovo International Airport,...,588.0,19016,Apache Air,0,734,B734,Boeing 737-400,B737,Boeing,Boeing
92357,United States,2912.0,Manas International Airport,Bishkek,Kyrgyzstan,43.061298,74.477600,2058.0,2913.0,Osh Airport,...,2927.0,19016,Apache Air,0,734,B734,Boeing 737-400,B737,Boeing,Boeing


In [362]:
sorted_df.to_csv('cleaned_data/cleaned.csv', index=False) 